<a href="https://colab.research.google.com/github/amarie-51/projekt_NLP/blob/main/hate_speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ** TASK 1**




Przetwarzanie wstępne 

In [ ]:
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import nltk
import re
import tensorflow as tf

nltk.download('punkt')



Tworzymy dataframe z treścią tweeta oraz oceną czy jest hejtem

In [2]:
tags_poprawne=pd.read_csv('/content/test_set_clean_only_tags.txt', header = None)
tweets=open('/content/test_set_clean_only_text.txt')
table_of_tweets=[]
for line in tweets.readlines():
  table_of_tweets.append(line)

tags_poprawne.columns=["Czy hate"]
tweets=pd.DataFrame(table_of_tweets)
tweets.columns=["Tweet"]
df_tweets=pd.concat([tweets,tags_poprawne],axis=1)



In [3]:
df_tweets.loc[:,"Tweet"] = df_tweets.Tweet.apply(lambda x : str.lower(x))
df_tweets.loc[:,"Tweet"] = df_tweets.Tweet.apply(lambda x : " ".join(re.findall('[\w]+',x)))

Dodać txt ze stop_words, które trzeba usunąć

Usuwam stop_words

In [4]:
with open("/content/stop_words.txt", "r") as stop_words:
	lines = stop_words.read().splitlines()


stop_words=lines
def remove_stopWords(s):
    '''For removing stop words
    '''
    s = ' '.join(word for word in s.split() if word not in stop_words)
    return s

df_tweets.loc[:,"Tweet"] = df_tweets.Tweet.apply(lambda x: remove_stopWords(x))


In [ ]:
df_tweets["Tokeny"]=df_tweets["Tweet"]
df_tweets['Tokeny'] = df_tweets.apply(lambda row: nltk.word_tokenize(row['Tweet']), axis=1)
print(df_tweets.columns)


Wizualizacja słów,
 w razie gdyby pojawiły się jeszcze słowa, które warto usunąć np. xd, dodaje do stop_words





In [ ]:
def rozklad_slow(data):
  all_words = ' '.join([text for text in data ])
  wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(all_words)
  plt.figure(figsize=(10, 7))
  plt.imshow(wordcloud, interpolation="bilinear")
  plt.axis('off')
  plt.show()
rozklad_slow(df_tweets["Tweet"])

sprawdzamy najczestsze słowa w tweetach pozytywnych oraz później w negatywnych

In [ ]:
df_negatywne=df_tweets.loc[df_tweets["Czy hate"]==1].reset_index()

rozklad_slow(df_negatywne['Tweet'])

df_negatywne_random=df_negatywne.sample(n=100)
df_negatywne=pd.concat([df_negatywne,df_negatywne,df_negatywne_random]) #robię trochę manualny oversampling, tworze nowy dataframe z podwojana liczba tweetów negatywnych
print(df_negatywne["Tweet"].head)

In [ ]:
df_pozytywne=df_tweets.loc[df_tweets["Czy hate"]==0].reset_index()

rozklad_slow(df_pozytywne["Tweet"])
print(df_pozytywne["Tweet"].head)
df_random_pozytywne=df_pozytywne.sample(n=600)#do nowego dataframe, downsampling tweetów negatywnych

In [ ]:
df_tweets=pd.concat([df_random_pozytywne, df_negatywne])

print(df_tweets["Czy hate"].head)


najpopularniejsze słowa wg rodzaju tweeta

In [ ]:
from collections import Counter
from collections import Counter
import matplotlib.pyplot as plt

def get_most_common(df,top=20):
  words=Counter(" ".join(df).split()).most_common(top)
  print(f'Found {len(words)} words.')
  most_common = Counter(words).most_common(top)
  labels = []
  values = []
  for label, val in most_common:
    labels.append(label)
    values.append(val)
  return values, labels

def plot_most_common(df, top=20, title=''):
  values, labels = get_most_common(df, top)
  x = range(len(values))

  plt.figure(figsize=(15, 5))
  plt.bar(x, values)
  plt.xticks(x, labels, rotation=90)
  plt.title(title)
  plt.show()

plot_most_common(tweets_train, top=30, title='Top 30 most common words in train set')

# lemmatyzacja

In [ ]:
!pip install --upgrade spacy

In [ ]:
!sudo python -m spacy download pl_core_news_sm

In [14]:
import spacy
from spacy.lang.pl.stop_words import STOP_WORDS as stop_words_spacy

nlp=spacy.load('pl_core_news_sm')

In [15]:
def lemmatize(comments: list, nlp: spacy) -> list:
  
  def is_redundant(token):
    return any([token.is_punct, token.is_stop, token.is_space])

  return [' '.join([token.lemma_ for token in nlp(comment) if not is_redundant(token)]) for comment in comments]

podział zbioru tweetów na dwa datasety

In [16]:
from sklearn.model_selection import train_test_split
tweets = df_tweets['Tweet'].values
label=df_tweets['Czy hate'].values

tweets_train, tweets_test, label_train, label_test = train_test_split(tweets, label, test_size=0.2, shuffle=True)


oversampling, downsampling

In [ ]:
!pip install imbalanced-learn

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler


# lematyzacja wytrenowanego modelu 
lematyzacja poprawiła recall i f1-score

In [19]:
tweets_test=lemmatize(tweets_test, nlp)
tweets_train = lemmatize(tweets_train, nlp)

In [ ]:
rozklad_slow(tweets_train) #rozkład słów poddanych lematyzacji

Wektoryzacja tekstu

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit(tweets_train)

X_train = vectorizer.transform(tweets_train)
X_test  = vectorizer.transform(tweets_test)
print(X_train.shape,X_test.shape)


regresja liniowa

In [21]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(X_train, label_train)
score = classifier.score(X_test, label_test)

print("Accuracy:", score)

Accuracy: 0.9536082474226805


# Sieci neuronowe 

In [30]:
import numpy as np
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Embedding,LSTM
import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt

In [29]:

model= Sequential([
                   Dense(units=16,activation="relu"),
                   Dropout(0.5),
                   Dense(units=8,activation="relu"),
                   Dense(units=1, activation="sigmoid")
                  
])
model.compile(loss="binary_crossentropy", optimizer="Adam", metrics=["accuracy"])


In [ ]:
model.build(X_train.shape)
print(model.summary())

In [ ]:
history= model.fit(X_train,label_train, validation_data=(X_test,label_test), batch_size=8, epochs=10)

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
tf.keras.backend.clear_session() #wyczysczenie wytrenowanego modelu

In [ ]:
model.evaluate(X_test,label_test)


In [ ]:
loss, accuracy = model.evaluate(X_train, label_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, label_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))


Wykres przedstawiajacy  loss oraz accuracy

In [34]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training accuracy')
    plt.plot(x, val_acc, 'r', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()


In [ ]:
plot_history(history) 

Sprawdzamy model

In [ ]:
y_pred=model.predict(X_test).reshape(-1)
print(y_pred[:20])

y_pred=np.round(y_pred)


In [ ]:
print(y_pred[:10])
print(label_test[:10])

Ponieważ zbiór danych nie jest równy, dlatego accuracy nie jest dobrym wyznacznikiem modelu.
Wartość recall oraz f1-score są niskie, co świadczy o słabym działaniu modelu.
Większość tweetów to sa jednak tweety neutralne.
Tylko 30% to tweety negatywne

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(label_test,y_pred))